한국어 전처리 패키지 (Text Preprocessing tools for Korean Text)
앞에서 공부했던 형태소와 문장 토크나이징 도구인 Konlpy와 KSS와 함께 사용하기 딱 좋다
PyKoSpacing 등이 있다.
우선 pip install git+https://github.com/haven-jeon/PyKoSpacing.git 를 사용해서 다운로드 받을 수 있다.
이 패키지는 띄어쓰기가 되어있지 않은 문장을 띄어쓰기 해준다. 대용량 코퍼스로 학습한 딥러닝 모델을 기반으로 해주기 때문에 정확하다.


In [1]:
sent = '김철수는 극중 두 인격의 사나이 이광수 역을 맡았다. 철수는 한국 유일의 태권도 전승자를 가리는 결전의 날을 앞두고 10년간 함께 훈련한 사형인 유연재(김광수 분)를 찾으러 속세로 내려온 인물이다.'
#이런 문장이 있을 때 우선 띄어쓰기를 제거해주자
new_sent = sent.replace(' ','')

from pykospacing import Spacing
spacing = Spacing()

kospacingsent = spacing(new_sent)
print(sent)
print(kospacingsent)

김철수는 극중 두 인격의 사나이 이광수 역을 맡았다. 철수는 한국 유일의 태권도 전승자를 가리는 결전의 날을 앞두고 10년간 함께 훈련한 사형인 유연재(김광수 분)를 찾으러 속세로 내려온 인물이다.
김철수는 극중 두 인격의 사나이 이광수 역을 맡았다. 철수는 한국 유일의 태권도 전승자를 가리는 결전의 날을 앞두고 10년간 함께 훈련한 사형인 유연재(김광수 분)를 찾으러 속세로 내려온 인물이다.


Py-hanspell은 네이버 한글 맞춤법 검사기를 통해서 만들어진 패키지이다.


In [2]:
from hanspell import spell_checker

sent = "맞춤법 틀리면 외 않되? 쓰고싶은대로쓰면돼지 "
spelled_sent = spell_checker.check(sent) # 출력결과를 보면 checked에 틀린것을 고친걸 저장시켜놨기 때문에
print(spelled_sent)
hanspell_sent = spelled_sent.checked # checked로 호출해서 수정된 버전을 출력한다.
print(hanspell_sent)

Checked(result=True, original='맞춤법 틀리면 외 않되? 쓰고싶은대로쓰면돼지 ', checked='맞춤법 틀리면 왜 안돼? 쓰고 싶은 대로 쓰면 되지 ', errors=2, words=OrderedDict([('맞춤법', 0), ('틀리면', 0), ('왜', 1), ('안돼?', 1), ('쓰고', 1), ('싶은', 1), ('대로', 1), ('쓰면', 1), ('되지', 1), ('', 0)]), time=0.08986186981201172)
맞춤법 틀리면 왜 안돼? 쓰고 싶은 대로 쓰면 되지 


In [3]:
new_spelled_sent = spell_checker.check(new_sent)
hanspell_sent2 = new_spelled_sent.checked
print(hanspell_sent2)

김철수는 극 중 두 인격의 사나이 이광수 역을 맡았다. 철수는 한국 유일의 태권도 전승자를 가리는 결전의 날을 앞두고 10년간 함께 훈련한 사형인 유연제(김광수 분)를 찾으러 속세로 내려온 인물이다.


위와 같이 hanspell은 띄어쓰기와 맞춤법 둘다 보정되는 것을 확인할 수 있다. 
다른 패키지로 SOYNLP를 통해서 진행할 수 있다.
진행하기 앞서서 기존 형태소 분석기들이 뭐가 문제인지, SOYNLP는 어떤 부분에서 유리한지 정리하자
기존 형태소 분석기들은 신조어나 형태소 분석기에 등록되어있지 않은 말들은 제대로 구분하지 못한다.

In [5]:
from konlpy.tag import Okt

tokenizer = Okt()
text = '에스파 카리나 ㄹㅇ 존예돌'
print(tokenizer.morphs(text))

['에스', '파', '카리나', 'ㄹㅇ', '존예', '돌']


위와 같이 존예돌 하나로 묶여야하는 것, 그리고 에스파 하나로 묶여야하는 것이 분리된 것을 확인할 수 있다.
그렇다면 이런 문제를 해결하려면 어떻게 해야할까?
특정 문자 시퀀스가 함께 자주 등장하는 빈도가 높고 앞뒤로 조사 또는 완전히 다른 단어들이 나오는 것을 형태소로 판단하는 토크나이저라면 어떨까?
이런 토크나이저가 SOYNLP이다.

In [6]:
import urllib.request
from soynlp import DoublespaceLineCorpus
from soynlp.word import WordExtractor
#soynlp는 기본적으로 학습에 기반한 토크나이저이므로 학습에 필요한 문서를 다운받는다.
urllib.request.urlretrieve("https://raw.githubusercontent.com/lovit/soynlp/master/tutorials/2016-10-20.txt", filename="2016-10-20.txt")


('2016-10-20.txt', <http.client.HTTPMessage at 0x170dcebc820>)

In [ ]:
#훈련데이터를 다수의 문서로 분리

corpus = DoublespaceLineCorpus('2016-10-20.txt')
len(corpus)

i = 0
for doc in corpus:
    if len(doc)>0:
        print(doc)
    i+=1
    if i == 0:
        break #일부분 출력해보기

In [9]:
#soynlp는 학습기반 토크나이저이므로 꼭 학습과정을 거쳐야한다.
word_extractor = WordExtractor()
word_extractor.train(corpus)
word_score_table = word_extractor.extract()



training was done. used memory 1.202 Gb
all cohesion probabilities was computed. # words = 223348
all branching entropies was computed # words = 361598
all accessor variety was computed # words = 361598


soynlp의 응집확률(cohesion probability)
응집 확률은 내부 문자열이 얼마나 응집해서 자주 등장하는지 나타낸다. 값이 높을수록 문자열 시퀀스는 하나의 단어로 등장할 확률이 높고 이를 판단해서 토크나이저를 하게 된다.

In [15]:
print(word_score_table["반포한"].cohesion_forward)
print(word_score_table["반포한강공원"].cohesion_forward)


0.08838002913645132
0.37891487632839754


soynlp의 브랜칭 엔트로피(branching entropy)
확률분포의 엔트로피값을 사용한다. 주어진 문자열에서 얼마나 다음 문자가 등장할 수 있는지 판단하는 척도이다. 
예를 들어서 '디'가 들어가면 다음 문자를 맞추기 어렵다.
'디스'가 들어가면 조금 더 명확해지지만 아직까진 감이 안오고
'디스플'이 되면 거의 '디스플레이'이다. 이를 판단할 수 있게 해준다.

In [17]:
print(word_score_table['디'].right_branching_entropy)
print(word_score_table['디스'].right_branching_entropy)
print(word_score_table['디스플'].right_branching_entropy)

2.68517802819071
1.6371694761537934
-0.0


soynlp L tokenizer 
한국어는 L토큰 + R토큰 의 형식을 가질 수 있다. 
공원에 -> 공원 + 에
공부하는 -> 공부 + 하는 
이런식으로 말이다. 분리기준을 점수로 표현하고 분리가 가장 잘될때를 찾는다.

또한 최대 점수 토크나이저는 띄어쓰기가 되어있지 않을때 점수가 높은 글자 시퀀스를 순차적으로 찾아낸다.

In [19]:
from soynlp.tokenizer import LTokenizer, MaxScoreTokenizer

scores = {word:score.cohesion_forward for word, score in word_score_table.items()}
l_tokenizer = LTokenizer(scores=scores)
l_tokenizer.tokenize("국제사회와 우리의 노력들로 범죄를 척결하자", flatten=False)

maxscore_tokenizer = MaxScoreTokenizer(scores=scores)
maxscore_tokenizer.tokenize("국제사회와우리의노력들로범죄를척결하자")

['국제사회', '와', '우리', '의', '노력', '들로', '범죄', '를', '척결', '하자']

soynlp를 통한 반복문자 정제
한국어에서는 ㅋㅋㅋㅋ ㅋㅋㅋ ㅋㅋ ㅋㅋㅋㅋㅋㅋㅋㅋ 와 같이 중복되는 경우가 많다. 이를 통합시키면 더 효율적으로 판단할 수 있을 것이다. 

In [20]:
from soynlp.normalizer import *
print(emoticon_normalize('앜ㅋㅋㅋㅋ이영화존잼쓰ㅠㅠㅠㅠㅠ',num_repeats=2))
print(emoticon_normalize('앜ㅋㅋㅋㅋㅋㅋㅋㅋㅋ이영화존잼쓰ㅠㅠㅠㅠㅠ',num_repeats=2))
print(emoticon_normalize('앜ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ이영화존잼쓰ㅠㅠㅠㅠㅠ',num_repeats=2))
print(emoticon_normalize('앜ㅋㅋㅋㅋㅋㅋㅋ이영화존잼쓰ㅠㅠㅠㅠㅠ',num_repeats=2))
print(emoticon_normalize('앜ㅋㅋㅋ이영화존잼쓰ㅠㅠㅠㅠㅠ',num_repeats=2))

아ㅋㅋ영화존잼쓰ㅠㅠ
아ㅋㅋ영화존잼쓰ㅠㅠ
아ㅋㅋ영화존잼쓰ㅠㅠ
아ㅋㅋ영화존잼쓰ㅠㅠ
아ㅋㅋ영화존잼쓰ㅠㅠ
